In [ ]:
# 라이브러리 설치
!pip install google-api-python-client

In [5]:
from googleapiclient.discovery import build
import pandas as pd

In [6]:
# API 정보 설정
API_KEY = 'YOUR_API_KEY' # 개인 API키입니다.
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

# YouTube API 클라이언트 생성
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY)

# YouTube 비디오 메타데이터 상세 정보

| 변수명 | 설명 | 예시 |
|--------|------|------|
| `video_id` | 비디오의 고유 식별자 | `"dQw4w9WgXcQ"` |
| `title` | 비디오의 제목 | `"Rick Astley - Never Gonna Give You Up (Official Music Video)"` |
| `channel_id` | 비디오를 업로드한 채널의 고유 식별자 | `"UCuAXFkgsw1L7xaCfnd5JJOw"` |
| `channel_title` | 비디오를 업로드한 채널의 이름 | `"Rick Astley"` |
| `category_id` | 비디오의 카테고리 ID | `"10"` (음악 카테고리) |
| `publish_date` | 비디오가 게시된 날짜와 시간 | `"2009-10-25T06:57:33Z"` |
| `duration` | 비디오의 길이 (ISO 8601 형식) | `"PT3M32S"` (3분 32초) |
| `view_count` | 비디오의 총 조회수 | `1234567890` |
| `like_count` | 비디오의 좋아요 수 | `12345678` |
| `comment_count` | 비디오의 총 댓글 수 | `123456` |
| `thumbnail_url` | 비디오 썸네일 이미지의 URL | `"https://i.ytimg.com/vi/dQw4w9WgXcQ/default.jpg"` |
| `tags` | 비디오에 할당된 태그들 (쉼표로 구분) | `"Rick Astley,Rick Roll,RickRoll,Never Gonna Give You Up,Rick Astley album"` |
| `description` | 비디오의 설명 텍스트 | `"The official video for 'Never Gonna Give You Up' by Rick Astley..."` |

In [18]:
def get_video_details(video_ids):
    video_data = []
    
    # YouTube API는 한 번에 최대 50개의 비디오 정보를 요청 가능
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response['items']:
            stats = video['statistics']
            snippet = video['snippet']
            content_details = video['contentDetails']
            
            video_data.append({
                'video_id': video['id'],
                'title': snippet['title'],
                'channel_id': snippet['channelId'],
                'channel_title': snippet['channelTitle'],
                'category_id': snippet['categoryId'],
                'publish_date': snippet['publishedAt'],
                'duration': content_details['duration'],
                'view_count': int(stats.get('viewCount', 0)),
                'like_count': int(stats.get('likeCount', 0)),
                'comment_count': int(stats.get('commentCount', 0)),
                'thumbnail_url': snippet['thumbnails']['default']['url'],
                'tags': ','.join(snippet.get('tags', [])),
                'description': snippet['description']
            })
    
    return video_data

# 실행 예시
# 비디오 아이디 찾는법: https://www.youtube.com/watch?v=dQw4w9WgXcQ # v= 이후 정보가 아이디
video_ids = ['dQw4w9WgXcQ', 'Y-JY4LC4S-4'] # 비디오 아이디 입력 (type: list)
video_data = get_video_details(video_ids)
df_videos = pd.DataFrame(video_data) # dataframe

df_videos.head()

,video_id,title,channel_id,channel_title,category_id,publish_date,duration,view_count,like_count,comment_count,thumbnail_url,tags,description
0,dQw4w9WgXcQ,Rick Astley - Never Gonna Give You Up (Officia...,UCuAXFkgsw1L7xaCfnd5JJOw,Rick Astley,10,2009-10-25T06:57:33Z,PT3M33S,1558847446,17758635,2350444,https://i.ytimg.com/vi/dQw4w9WgXcQ/default.jpg,"rick astley,Never Gonna Give You Up,nggyu,neve...",The official video for “Never Gonna Give You U...
1,Y-JY4LC4S-4,"[뉴스 '꾹'] ""화살 어디갔죠?"" 중계진 당황 그럼에도 '찬사' 쏟아진 이유 (2...",UCF4Wxdo3inmxP-Y59wXDsFw,MBCNEWS,25,2024-07-31T03:17:39Z,PT2M36S,357070,7441,1018,https://i.ytimg.com/vi/Y-JY4LC4S-4/default.jpg,"MBC,MBC뉴스,뉴스데스크,newsdesk,뉴스투데이,newstoday,8시뉴스,...","#양궁 #1점 #차드 #파리올림픽 \nㅤ\nⓒ MBC & iMBC 무단 전재, 재배..."


# YouTube 댓글 메타데이터 상세 정보

| 변수명 | 설명 | 예시 |
|--------|------|------|
| `video_id` | 댓글이 달린 비디오의 고유 식별자 | `"dQw4w9WgXcQ"` |
| `comment_id` | 댓글의 고유 식별자 | `"UgxYYYYYYYYYYYYYYYYYYAaABAg"` |
| `author` | 댓글 작성자의 표시 이름 | `"John Doe"` |
| `comment_text` | 댓글의 실제 내용 | `"This video never gets old!"` |
| `like_count` | 해당 댓글의 좋아요 수 | `1234` |
| `publish_date` | 댓글이 게시된 날짜와 시간 | `"2023-07-31T12:34:56Z"` |날짜와 시간

In [19]:
def get_comments(video_id, max_results=100):
    comments = []
    try:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=max_results,
            textFormat="plainText"
        )
        response = request.execute()
        
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append({
                'video_id': video_id,
                'comment_id': item['id'],
                'author': comment['authorDisplayName'],
                'comment_text': comment['textDisplay'],
                'like_count': comment['likeCount'],
                'publish_date': comment['publishedAt']
            })
            
    except HttpError as e:
        print(f"An error occurred: {e}")
    
    return comments

video_id = 'dQw4w9WgXcQ'
comments = get_comments(video_id)
df_comment = pd.DataFrame(comments)

df_comment.head(3)

,video_id,comment_id,author,comment_text,like_count,publish_date
0,dQw4w9WgXcQ,UgzarqjaaPC7TbFINNx4AaABAg,@RickAstleyYT,1 BILLION views for Never Gonna Give You Up! ...,1390932,2021-07-28T21:00:32Z
1,dQw4w9WgXcQ,UgxECFo6-w0Gmyj9Zzp4AaABAg,@partharora4465,chat gpt rickrolled me,0,2024-07-31T06:23:21Z
2,dQw4w9WgXcQ,UgxDJ7YDlNrxmudmMq54AaABAg,@thegameface,Меня зарикролили.,0,2024-07-31T06:12:44Z
